<a href="https://colab.research.google.com/github/mamikah/DeepLearning2022Late/blob/main/%E5%8F%8E%E9%9B%86%E3%81%97%E3%81%9F%E7%94%BB%E5%83%8F%E3%81%A7%E5%AD%A6%E7%BF%92%E3%81%A8%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 収集した画像をColabにUPして学習させ、それをもとに分類させる

## ファイルのフォルダを準備する

In [7]:
import os
# 分類対象の画像のフォルダを作成
os.makedirs('sample',exist_ok = True)

# 犬と判定されたときの置き場所
os.makedirs('output/dog/sample',exist_ok = True)
# 猫と判定されたときの置き場所
os.makedirs('output/cat/sample',exist_ok = True)

# 犬の学習用の収集画像
os.makedirs('img/deep_learning/dog',exist_ok = True)
# 猫の学習用の収集画像
os.makedirs('img/deep_learning/cat',exist_ok = True)

## 必要なライブラリをimport

In [8]:
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

## 分類（クラス）の名前をフォルダ名（dog,cat）から引用する

In [9]:
# dogとcatフォルダがある場所
path = "img/deep_learning" 
folders = os.listdir(path)
# 2つのパスからフォルダ名を抜きだす
classes = [f for f in folders if os.path.isdir(os.path.join(path,f))]
print(classes)
n_classes = len(classes)
print(n_classes)

['dog', 'cat']
2


## 画像を読み込み、リサイズ・整形する

In [10]:
# 画像とラベルのための配列
X = []   # 画像（学習データの配列）
Y = []   # ラベル（正解用の配列）

# 画像を読みつつリサイズする
for label,class_name in enumerate(classes):
  files = glob.glob(path + "/" + class_name + "/*jpg")
  for file in files:
    img = cv2.imread(file)
    img = cv2.resize(img,dsize=(224,224))
    X.append(img)
    Y.append(label)

## 0～255のビットデータ値を、学習のために0～1に変換（正規化）
## 学習精度を上げる

In [11]:
# 配列XをNumpy配列に変換
X = np.array(X)
# 小数型に変換
X = X.astype('float32')
X /= 255.0
# ラベルもNumpyの配列に変換
Y = np.array(Y)
Y = np_utils.to_categorical(Y,n_classes)

## 整形したデータを学習用とテスト検証用に振り分ける

In [12]:
# 学習データ8割、テスト検証用が2割に分けられる
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2)
# 学習データ8割(画像問題データ)
print(X_train.shape)
# テストデータ2割(画像問題データ)
print(X_test.shape)

# 学習データ8割(正解ラベル)
print(Y_train.shape)
# テストデータ2割(正解ラベル)
print(Y_test.shape)

(120, 224, 224, 3)
(31, 224, 224, 3)
(120, 2)
(31, 2)


## 準備したデータを学習させていく
## 学習に必要なライブラリをimport

In [13]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential,Model
from keras.layers import Input,Activation,Dense,Flatten,Dropout
from keras.optimizers import Adam

## モデルのVGG16をアレンジしていく

In [14]:
# VGG16入力層を用意する（定義）
input_tensor = Input(shape=(224,224,3))
#VGG16のインスタンスを作る
base_model = VGG16(weights='imagenet',input_tensor=input_tensor,include_top=False)

58889256/58889256 [==============================] - 3s 0us/step


## 1000分類のsoftmaxを外した代わりに、二つに分類するsoftmax層を追加する

In [15]:
# 追加する層の作成
top_model = Sequential() # ラッピングする層
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes,activation='softmax'))

## 最後のsoftmaxを外したモデル（base_model）と追加用の二つに分類するsoftmax層のモデル(top_mode)をつなげる

In [16]:
model = Model(inputs=base_model.input,outputs=top_model(base_model.output))

## 時間短縮のために、15層までを外す

In [17]:
## ループしながら15層までを無効にする
for layer in model.layers[:15]:
  layer.trainable = False #15層までは学習しない
print('# layers =', len(model.layers))#モデルの層の数を表示

# layers = 20


## 学習モデルをコンパイルする

In [18]:
## 損失はクロスエントロピー法で算出、最適化はAdam、指標は精度で見る
model.compile(loss='categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

## compileしたモデルを表示

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## コンパイルしたモデルで収集した画像データを学習させる

In [20]:
model.fit(X_train,Y_train,epochs=20,batch_size=16)

Epoch 1/20
8/8 [==============================] - 12s 230ms/step - loss: 1.3380 - accuracy: 0.5417
Epoch 2/20
8/8 [==============================] - 1s 86ms/step - loss: 0.7697 - accuracy: 0.5667
Epoch 3/20
8/8 [==============================] - 1s 85ms/step - loss: 0.7068 - accuracy: 0.4917
Epoch 4/20
8/8 [==============================] - 1s 85ms/step - loss: 0.6968 - accuracy: 0.5083
Epoch 5/20
8/8 [==============================] - 1s 85ms/step - loss: 0.6934 - accuracy: 0.5083
Epoch 6/20
8/8 [==============================] - 1s 84ms/step - loss: 0.6931 - accuracy: 0.5083
Epoch 7/20
8/8 [==============================] - 1s 86ms/step - loss: 0.6931 - accuracy: 0.5083
Epoch 8/20
8/8 [==============================] - 1s 85ms/step - loss: 0.6931 - accuracy: 0.5083
Epoch 9/20
8/8 [==============================] - 1s 85ms/step - loss: 0.6931 - accuracy: 0.5083
Epoch 10/20
8/8 [==============================] - 1s 86ms/step - loss: 0.6931 - accuracy: 0.5083
Epoch 11/20
8/8 [==========

## 学習の成果を指標（accuracy）で確認

In [21]:
score = model.evaluate(X_test,Y_test,batch_size=16)

2/2 [==============================] - 2s 2s/step - loss: 0.6940 - accuracy: 0.4516


## 作成した学習モデルとクラス分類名を保存

In [22]:
import pickle
# オブジェクトをバイナリファイルで保存
pickle.dump(classes,open('classes.sav','wb'))
# 学習モデルの保存
model.save('cnn.h5')

## 保存したクラス分類と学習モデルを読み込んで使う

In [23]:
# ライブラリのimport
from keras.models import load_model
import pickle
import cv2
import glob

## ファイルを読み込んでオブジェクトとして蘇らせる

In [24]:
model = load_model('cnn.h5')
classes = pickle.load(open('classes.sav','rb'))

## sample画像を読み込む

In [25]:
files = glob.glob('sample/*')
print(files)

['sample/cat_8.jpg', 'sample/dog_6.jpg', 'sample/cat_13.jpg']


## ファイルパスから画像を読み込み、学習モデルに判定させ、該当するoutputフォルダにコピーを書き込む

In [32]:
for file in files: # ファイルの数だけループ
  img = cv2.imread(file) # 画像データを読み込み
  # 判定のためにimgを加工したimg2を作る
  img2 = cv2.resize(img,dsize=(224,224)) #b 224,224にリサイズ
  img2 = img2.astype('float32')
  img2 = img2/255.0 #0~255.0だったデータを0.0~0.1に変換
  img2 = img2[None,...] # 1次元配列に変換
  # 正規化したデータを判定
  result = model.predict(img2)
  print(result) # 画面に生で出力
  # 確率が高いクラスを取得
  pred = result.argmax()
  print(pred , str(classes[pred]))
  # 判定したクラス分類のフォルダの方に、画像データを書き込む
  cv2.imwrite('output/' + str(classes[pred]) + '/' + file,img)

1/1 [==============================] - 0s 24ms/step
[[0.49577782 0.5042222 ]]
1 cat
1/1 [==============================] - 0s 34ms/step
[[0.49577782 0.5042222 ]]
1 cat
1/1 [==============================] - 0s 35ms/step
[[0.49577782 0.5042222 ]]
1 cat
